In [1]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
import numpy as np
import io
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-jginx629
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-jginx629
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 4474e37c425940d71b0a7747af76c2e12e8c44bb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached pathlib2-2.3.6-py2.py3-none-any.whl (17 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached DateTime-4.3-py2.py3-none-any.whl (60 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached re

In [2]:
# Test 1 : search_sirene with number parameters with pynsee

variable = ["activitePrincipaleEtablissement"]
pattern = ['47.22Z']

donnees_pynsee = search_sirene(variable = variable, pattern = pattern, kind="siret", number=21000, query_limit=41)
donnees_pynsee.shape

Getting legal entities: 100%|██████████| 24/24 [00:04<00:00,  5.82it/s]



!!! This function may return personal data, please check and
 comply with the legal framework relating to personal data protection !!!


(7745, 82)

In [25]:
# Test 2 : iterative search_sirene with pynsee


cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
liste_dep= cog_commune["dep"].dropna().unique()

donnees_pynsee_iterative=pd.DataFrame()

variable = ["activitePrincipaleEtablissement","codeCommuneEtablissement"]
for _,code_departement in enumerate(liste_dep):
    pattern = ['47.22Z']
    pattern.append(code_departement+"*")
    df_temp=search_sirene(variable = variable, pattern = pattern, kind="siret", number=41000)
    donnees_pynsee_iterative=donnees_pynsee_iterative.append(other=df_temp,ignore_index=True)
    
donnees_pynsee_iterative.shape

Data will be stored in the following location: /tmp/tmp28u82me6


Downloading: : 1.93MiB [00:01, 1.02MiB/s]                         
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:182: UserWarning: File in insee.fr modified or corrupted during download
  warnings.warn("File in insee.fr modified or corrupted during download")


No destination directory defined. Data have been written there: /tmp/tmpkxuao3me


Extracting: 100%|██████████| 9.00M/9.00M [00:00<00:00, 138MB/s]
Getting legal entities: 100%|██████████| 2/2 [00:00<00:00, 29330.80it/s]


(21244, 82)

In [6]:
# Test 3 : Insee sirene data  by sirene.fr website
#from IPython.display import display,HTML
session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)
donnees_sirene_fr.shape

(20174, 76)

In [78]:
# comptage par dep metro

comptage_pynsee_metro=donnees_pynsee[donnees_pynsee.codeCommuneEtablissement.str[:2]!='97']["codeCommuneEtablissement"].groupby(donnees_pynsee.codeCommuneEtablissement.str[:2]).count().to_frame().rename({"codeCommuneEtablissement":"comptagePynsee"}, axis=1).reset_index()
comptage_pynsee_iterative_metro=donnees_pynsee_iterative[donnees_pynsee_iterative.codeCommuneEtablissement.str[:2]!='97']["codeCommuneEtablissement"].groupby(donnees_pynsee_iterative.codeCommuneEtablissement.str[:2]).count().to_frame().rename({"codeCommuneEtablissement":"comptagePynseeIterative"}, axis=1).reset_index()
comptage_sirene_metro=donnees_sirene_fr[donnees_sirene_fr.codeCommuneEtablissement.str[:2]!='97']["codeCommuneEtablissement"].groupby(donnees_sirene_fr.codeCommuneEtablissement.str[:2]).count().to_frame().rename({"codeCommuneEtablissement":"comptageSireneFr"}, axis=1).reset_index()

comptage_pynsee_metro.merge(comptage_pynsee_iterative_metro).merge(comptage_sirene_metro)


,codeCommuneEtablissement,comptagePynsee,comptagePynseeIterative,comptageSireneFr
0,01,70,229,214
1,02,51,124,121
2,03,39,112,106
3,04,33,80,79
4,05,28,74,70
...,...,...,...,...
91,91,85,312,297
92,92,130,399,382
93,93,160,629,594
94,94,109,382,360
